In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import hvplot.xarray
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# If you want a notebook representation of the xarray datasets
# with nested attributes, you can use the xopr accessor like this:L
# > ds.xopr
import xopr.xarray_repr_fix.xopr_accessor
from xopr.matlab_attribute_utils import decode_matlab_variable, dereference_h5value

import xopr.opr_access as xopr

hvplot.extension('bokeh')

import fsspec
import h5py

In [2]:
# OLD version -- use the accessor instead
# from xopr.xarray_monkeypatch import formatting_html
# xr.Dataset._repr_html_ = formatting_html.dataset_repr

In [3]:
# Establish an OPR session
# You'll probably want to set a cache directory if you're running this locally to speed
# up subsequent requests. You can do other things like customize the STAC API endpoint,
# but you shouldn't need to do that for most use cases.
opr = xopr.OPRConnection(cache_dir="radar_cache")

# Or you can open a connection without a cache directory (for example, if you're parallelizing
# this on a cloud cluster without persistent storage).
#opr = xopr.OPRConnection()

In [4]:
season, flight_id = '2022_Antarctica_BaslerMKB', '20230109_01'
#season, flight_id = '2016_Antarctica_DC8', None
print(f"Selected flight: {flight_id} from season {season}")

stac_items = opr.query_frames(seasons=[season], flight_ids=[flight_id], max_items=1)
frames = opr.load_frames(stac_items)

Selected flight: 20230109_01 from season 2022_Antarctica_BaslerMKB
Found 1 frames matching the query criteria


In [5]:
frames[0].xopr

# IGNORE BELOW

In [ ]:
source_url = frames[0].attrs['source_url']
file = fsspec.open_local(f"{opr.fsspec_url_prefix}{source_url}", filecache=opr.fsspec_cache_kwargs)

In [ ]:
xr.open_datatree(file, engine='h5netcdf', phony_dims='sort')

In [ ]:
ds = xr.open_dataset(file, engine='h5netcdf', phony_dims='sort')
ds

h5pyfile = h5py.File(file, 'r')

ds.attrs['properties'] = decode_matlab_variable(h5pyfile, skip_variables=True, skip_errors=True)

ds.xopr

In [ ]:
r = h5pyfile['param_array/array_proc/fcs/filter'][:].flat[0]
isinstance(h5pyfile[r]['function_handle'], Iterable)

In [ ]:
#filter at /param_array/array_proc/fcs

dereference_h5value(h5pyfile['param_array/array_proc/fcs/filter'][:], h5file=h5pyfile)

In [ ]:
from collections.abc import Iterable

#hidden_depend_funs at /param_array/cluster

#isinstance(h5pyfile['param_array/cluster']['hidden_depend_funs'][:], Iterable)

print(h5pyfile['param_array/array/imgs'].attrs['MATLAB_class'])
r = h5pyfile['param_array/array/imgs'][:].flat[0]
r2 = h5pyfile[r][:].flat[0]
h5pyfile[r2][:]


In [ ]:
h5pyfile[r2].name

In [ ]:
h5pyfile['param_array/array/imgs'].shape

In [ ]:
res = dereference_h5value(h5pyfile['param_array/array/imgs'][:], h5file=h5pyfile)
print(res)
res_numpy = np.array(res)
print(np.shape(res))

In [ ]:
#/param_array/array_proc/fcs
ref = h5pyfile['param_array']['array_proc']['fcs']['filter'][:].flat[0]
#h5pyfile[ref]['function_handle']['file']
for k in h5pyfile[ref]['function_handle'].keys():
    print(dereference_h5value(h5pyfile[ref]['function_handle'][k], h5file=h5pyfile))
h5pyfile[ref]['function_handle']

In [ ]:
decode_matlab_variable(h5pyfile['param_array']['array_proc']['fcs'])

In [ ]:
vectorized_dereference = np.vectorize(lambda x: dereference_h5value(x, h5pyfile))

vectorized_dereference(h5pyfile['param_records']['array']['imgs'][:])

In [ ]:
ref = h5pyfile['param_records']['array']['imgs'][0][0]
#h5pyfile[ref][:]

In [ ]:
import h5netcdf

h5file = h5netcdf.File(file, 'r', phony_dims='sort')
h5file #.groups['param_array']

In [ ]:
for k in h5file.groups['param_records']:
    #print(k)
    #print(type(h5file.groups['param_records'][k]))
    if isinstance(h5file.groups['param_records'][k], h5netcdf.Variable):
        print(k)
        print(h5file.groups['param_records'][k].attrs)
        print(h5file.groups['param_records'][k][...])